In [ ]:
from funzioni import load_vector_from_file,solve_ab,Matrici_aerodinamiche,read_airfoil_file
import numpy as np
import sympy as sp
from scipy.optimize import minimize
import math
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

# funzione che dato un vettore di reali restuisce uno stato quantistico = codifica in ampiezze del vettore dato
def amplitude_encoding(vettore) : 
    norma = (sum([x**2 for x in vettore]))**(1/2) # somma dei quadrati (alfa in qiskit)
    # abbiamo un vettore di N elementi, 
    # vogliamo scrivere il vettore come comb lin di N vettori di base i cui coeff sono le componenti del vettore
    # quindi vogliamo costruire un circuito con n = math.ceil(log2(N)) bit 
    N = len(vettore)
    n = math.ceil(math.log2(N))
    # riempio le componenti fino a 2^N con zeri se serve (cioè se N non è una potenza di 2)
    for i in range(N,2**n) : # riempio da N a 2n-1
        vettore.append(0)
    # desired_state è |phi> che rappresenta vettore
    # (es : vettore=[4,8,5] e desired_state = [4/alfa,8/alfa,5/alfa,0] (abbiamo aggiunto 0 prima) )
    desired_state = [x/norma for x in vettore]
    return desired_state

# funzione di costo
def cost (sigma_0, teta, rhs, vn_ext) :

    sigma = solve_ab(teta[0], teta[1], vn_ext, rhs) 
    sigma = amplitude_encoding(list(sigma)) # lo casto a list cosi posso usare append (sigma è np.ndarray)
    sigma_0 = amplitude_encoding(list(sigma_0))
    # non normalizzo come nel caso classico perchè sigma_0 e sigma sono gia normalizzati in amplitude encoding
    #(al circuito dovrei dare sigma e sigma_0 normalizzati come stati quantistici)
    # invece di fare 1 - mod_quadro come nel caso classico, faccio lo swaptest

    ######################################### CIRCUITO QUANTISTICO
    # il circuito è formato è un registro di 1 qubit, un secondo di n qubit e un terzo di n qubit
    # (n è lo stesso perchè sigma e sigma_0 hanno stessa lunghezza e questi due registri li rappresentano,
    # il primo è per il controllo e la misura)
    N = len(sigma_0) 
    n = math.ceil(math.log2(N))
    qc = QuantumCircuit(2*n+1,1)
    qc.h(0)
    for i in range(N) :
        sigma[i] = float(sigma[i])
        sigma_0[i] = float(sigma_0[i])
    qc.initialize(sigma_0,range(1,n+1))
    qc.initialize(sigma,range(n+1,2*n+1))
    for i in range(1,n+1) : # ho da 1 a 2n-esimo riga del circuito
        qc.cswap(0,i,i+n)
    qc.h(0) 
    qc.measure(0,0)
    result = []
    # print(qc)
    
    ################################### SIMULATORE
    # simulatore
    simulator = AerSimulator(method='statevector') # NON facciamo traspilazione
    # risultato di una misurazione (non importa fare shot perchè gia conosco la prob dallo statevector)
    nshots = 2000
    result = simulator.run(qc, shots=nshots).result() 
    counts = result.get_counts() 
    #print("Counts: ",counts)

    num_measure_0 = counts['0'] # numero di volte che il circuito misura 1

    loss = 2 -(2.0/nshots)*(num_measure_0) # cercata su : https://quantumcomputinguk.org/tutorials/introduction-to-the-swap-test-in-qiskit-with-code
    #print("Squared Inner Product:",str(s))
    # quando misura 0 sono uguali, quindi vogliamo che la probabilità che misuri 1 sia vicina a 0 

    return loss # cambia 


In [ ]:
# Simboli parametri
alpha_deg = 4
a_val=0.5
b_val=0.5
alpha = np.deg2rad(alpha_deg)
alpha_sym = sp.Float(alpha)# %%
# === Carica il vettore di riferimento ===
# sigma_0 = load_vector_from_file("siga0.txt") # è un vettore di reali
sigma_0 = np.array([
    [0.7815136080484366],
    [0.8702848948594854],
    [1.333594725296618],
    [1.656720020972868],
    [0.5612358696279389],
    [-0.2357639965490963],
    [-0.7068146733682219],
    [-1.059680396687932],
    [0.1033718030071634]
])

# === Carica i dati del profilo alare ===
data = read_airfoil_file("naca0012_07_sym.txt")
#data = read_airfoil_file("nomefile.dat")
vn_ext, rhs, *_ = Matrici_aerodinamiche(data, alpha_sym) # matrici simboliche :
# vn_ex è una matrice 9x9 in funzione delle componenti di teta (a,b), rhs vett analogo di dim 9 
# (in funzione di a e b perchè sono contenute in data che è un matrice di tutti elementi simbolici)

def wrapper(valore) :
    return cost(sigma_0, valore, rhs, vn_ext)

# cost(sigma_0, [a_val,b_val], rhs, vn_ext)

# Minimizzazione con Nelder-Mead
#%pip install scikit-optimize
from skopt import gp_minimize
from skopt.space import Real
result = gp_minimize(func = wrapper, dimensions = [Real(0.0001,1,name = 'x0' ),Real(0.0001,1,name = 'x1' )], n_calls = 100,n_initial_points = 10,acq_func = 'EI',random_state=42,verbose=True)
# result = gp_minimize(func = wrapper, [a_val,b_val], method='BFGS',verbose = True )
# bounds=[(0.00001,1),(0.00001,1)]
print("BFGS:", result.x, result.fun)




[[ 0.78151361]
 [ 0.87028489]
 [ 1.33359473]
 [ 1.65672002]
 [ 0.56123587]
 [-0.235764  ]
 [-0.70681467]
 [-1.0596804 ]
 [ 0.1033718 ]]
[(1.00000000000000, -0.00126000000000000), (0.378084300000000, -b), (0.0977959500000000, -a), (0.0116830100000000, -0.0183430500000000), (0.0, 0.0), (0.0116830100000000, 0.0183430500000000), (0.0977959500000000, a), (0.378084300000000, b), (1.00000000000000, 0.00126000000000000)]
Iteration No: 1 started. Evaluating function at random point.
N : len vettore :  9
N : len vettore :  9
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.8474
Function value obtained: 0.8680
Current minimum: 0.8680
Iteration No: 2 started. Evaluating function at random point.


C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma[i] = float(sigma[i])
C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma_0[i] = float(sigma_0[i])


N : len vettore :  9
N : len vettore :  9
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.8378
Function value obtained: 0.8730
Current minimum: 0.8680
Iteration No: 3 started. Evaluating function at random point.
N : len vettore :  9
N : len vettore :  9
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.8524
Function value obtained: 0.8750
Current minimum: 0.8680
Iteration No: 4 started. Evaluating function at random point.
N : len vettore :  9
N : len vettore :  9
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.8263
Function value obtained: 0.8130
Current minimum: 0.8130
Iteration No: 5 started. Evaluating function at random point.
N : len vettore :  9
N : len vettore :  9
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.7720
Function value obtained: 0.9290
Current minimum: 0.8130
Iteration No: 6 started. Evaluating function at random point.
N : len vettore :  9
N : len vettore :  9
Iteration No: 6 en

C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma[i] = float(sigma[i])
C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma_0[i] = float(sigma_0[i])


Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 1.6113
Function value obtained: 0.8360
Current minimum: 0.6870
Iteration No: 12 started. Searching for the next optimal point.
N : len vettore :  9
N : len vettore :  9


...

Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 1.4794
Function value obtained: 0.7940
Current minimum: 0.0010
Iteration No: 99 started. Searching for the next optimal point.
N : len vettore :  9
N : len vettore :  9


C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma[i] = float(sigma[i])
C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma_0[i] = float(sigma_0[i])


Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 1.7832
Function value obtained: 0.8440
Current minimum: 0.0010
Iteration No: 100 started. Searching for the next optimal point.
N : len vettore :  9
N : len vettore :  9


C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma[i] = float(sigma[i])
C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma_0[i] = float(sigma_0[i])


Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 1.4908
Function value obtained: 0.0010
Current minimum: 0.0010
BFGS: [0.05424432354738832, 0.0827567845681933] 0.0009999999999998899


In [13]:
a_val = 0.5
b_val = 0.5
print(cost(sigma_0, [a_val,b_val], rhs, vn_ext))
a_val = 0.7
b_val = 0.2
print(cost(sigma_0, [a_val,b_val], rhs, vn_ext))
a_val = 2
b_val = 4
print(cost(sigma_0, [a_val,b_val], rhs, vn_ext))
a_val = 0.05
b_val = 0.07
print(cost(sigma_0, [a_val,b_val], rhs, vn_ext))

print('prova : ',cost(sigma_0, [0.05475272, 0.07565386], rhs, vn_ext))

N : len vettore :  9
N : len vettore :  9
0.7070000000000001


C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma[i] = float(sigma[i])
C:\Users\cater\AppData\Local\Temp\ipykernel_19588\4022787431.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  sigma_0[i] = float(sigma_0[i])


N : len vettore :  9
N : len vettore :  9
0.903
N : len vettore :  9
N : len vettore :  9
0.8899999999999999
N : len vettore :  9
N : len vettore :  9
0.006000000000000005
N : len vettore :  9
N : len vettore :  9
prova :  0.0
